# BITS F312 - Neural Network and Fuzzy Logic



In [ ]:
# importing libraries required
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# connecting gdrive to access the datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# finding out current working directory
!pwd

/content/drive/MyDrive/NNFL/Data_A1


In [ ]:
# changing directory to - 'drive/MyDrive/NNFL/Data_A1/'
%cd drive/MyDrive/NNFL/Data_A1/

/content/drive/MyDrive/NNFL/Data_A1


In [ ]:
# defining plotting style
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (14, 14)

In [ ]:
def metrics(Y_true, Y_pred):
    FP=0 # For counting the False Positives
    FN=0 # For counting the False Negatives
    TN=0 # For counting the True Negatives
    TP=0 # For counting the True Positives

    for i in range(len(Y_true)):
      if Y_true[i]==1:
        if Y_pred[i]==1:
          TP+=1
        else:
          FN+=1
      else:
        if Y_pred[i]==0:
          TN+=1
        else:
          FP+=1

    print('--------------------------------------------------------------------------------')
  
    print("False Positives : ", FP)
    print("False Negatives : ", FN)
    print("True Negatives  : ", TN)
    print("True Positives  : ", TP)

    print('--------------------------------------------------------------------------------')

    sens= TP/(TP+FN)
    spes = TN/(TN+FP)

    print("Sensitivity : ", sens)
    print("Specificity : ", spes)
    print("Accuracy ((TN+TP)/(TN+TP+FN+FP)) : ", ((TP+TN)/(TN+FN+TP+FP)))
    print('--------------------------------------------------------------------------------')



In [ ]:
def normalize(data):
  normalized_vec = data

  if(len(data.shape) == 1):
    mean = np.mean(data[:])
    std_dev = np.std(data[:])
    normalized_vec[:] = (normalized_vec[:] - mean)/std_dev
  else:
    for i in range(1,data.shape[1]):
      mean = np.mean(data[:, i])
      std_dev = np.std(data[:, i])
      normalized_vec[:,i] = (normalized_vec[:,i] - mean)/std_dev
  
  return normalized_vec

#Q10
Implement the Maximum likelihood (ML) decision rule for the multiclass
classification task. Use the hold-out cross-validation approach (70% training and 30%
testing) for the selection of training and test instances of the ML classifier. You must
use the dataset data_q6_q7.txt for this question. Evaluate individual accuracy and
overall accuracy of ML classifier.

In [ ]:
def maximumLHood(filename):
  df_data = pd.read_csv(filename,sep="\t", header = None, engine='python')
  data = np.array(df_data)
  X = data[:,:-1]
  Y = data[:,-1]
  Y = Y - 1
  indices = np.array(range(0,X.shape[0]))
  val_range = int(0.7 * X.shape[0])
  np.random.shuffle(indices)
  train_val = indices[ :val_range]
  test_val = indices[val_range: ]
  X_train, X_test = X[train_val], X[test_val]
  Y_train, Y_test = Y[train_val], Y[test_val]
  
  categories = 3

  Y_test = list(Y_test)
  Y_dist_pred = [Y_test.count(y) for y in list(set(Y_test))]
  Y_dist_correct = [0]*categories
  mean_val = []
  covariance_val = []

  for i in range(categories):
    temp_X = []
    for j in range(X_train.shape[0]):
      if (Y_train[j]==i):
        temp_X.append(X_train[j])
    temp_X = np.array(temp_X)
    mean_val.append(np.mean(temp_X, axis = 0))
    covariance_val.append(np.cov(temp_X.T))

  correct = 0

  for j in range(X_test.shape[0]):
    LHood = [np.exp(-0.5*(np.transpose(X_test[j] - mean_val[i]).dot(np.linalg.inv(covariance_val[i])).dot(X_test[j] - mean_val[i]))) for i in range(categories)]
    y_pred = LHood.index(max(LHood))

    if (y_pred==Y_test[j]):
      Y_dist_correct[y_pred] = Y_dist_correct[y_pred] + 1
      correct = correct + 1

  for i in range(categories):
    accuracy_class = (Y_dist_correct[i]*100/Y_dist_pred[i])
    print("Accuracy for class {0} : {1}%".format(i+1, round(accuracy_class)))

  print("Overall Accuracy for the model is {0}%".format(round(correct*100/len(X_test))))


In [ ]:
maximumLHood('data_q6_q7.txt')

Accuracy for class 1 : 95%
Accuracy for class 2 : 95%
Accuracy for class 3 : 100%
Overall Accuracy for the model is 97%
